In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import torch
from torch import nn

In [4]:
from src.networks.skip_nets.modulated_additive_skip_net import ModulatedAdditiveSkipNet
from src.networks.core.net import Net
from src.networks.skip_nets.modulated_dense_skip_net import ModulatedDenseSkipNet
from src.networks.core.parallel_net import ParallelNet
from src.module_analysis import count_parameters
from src.networks.skip_nets.dense_skip_net import DenseSkipNet
from src.networks.core.layer_connections import LayerConnections
from src.networks.skip_nets.additive_skip_net import AdditiveSkipNet

def create_mod_dense(from_idx: int, to_idx: int, num_f: int):
    if from_idx != to_idx:
        return Net.as_net(create_ones_linear(num_f, 2))
    else:
        return Net.as_net(nn.Identity())

def create_mod_additive(from_idx: int, to_idx: int, num_f: int):
    if from_idx != to_idx:
        return Net.as_net(create_ones_linear(num_f, num_f))
    else:
        return Net.as_net(nn.Identity())

def create_ones_linear(in_features, out_features):
    l = nn.Linear(in_features, out_features)
    l.weight.data[:] = 1.0
    l.bias.data[:] = 0.0
    return l

def create_layer(i, is_last, in_features, out_features):
    return nn.Sequential(
        create_ones_linear(in_features, out_features),
        nn.CELU() if not is_last else nn.Identity(),
    )

num_features=3
# net = AdditiveSkipNet.from_layer_provider(
#     layer_provider=create_layer,
#     num_layers=4,
#     num_features=num_features,
#     layer_connections='full',
#     initial_direct_connection_weight=1.0,
#     initial_skip_connection_weight=0.01,
#     return_dense=False,
#     weights_trainable=False,
# )
# net = DenseSkipNet.from_layer_provider(
#     layer_provider=create_layer,
#     num_layers=10,
#     num_features=num_features,
#     layer_connections='full'
# )
# net = ParallelNet.from_layer_provider(create_layer, in_size=num_features, out_sizes=[5, 7, 11, 13])
# net = ModulatedDenseSkipNet.from_layer_provider(
#     layer_provider=create_layer,
#     modulator_provider=create_mod_dense,
#     in_size=num_features,
#     layer_out_sizes=[11, 23, 17, 13],
# )
net = ModulatedAdditiveSkipNet.from_layer_provider(
    layer_provider=create_layer,
    modulator_provider=create_mod_additive,
    num_features=num_features,
    num_layers=4,
    # return_dense=True,
)
print(net.count_parameters())
print(net)

out = net(torch.ones(num_features))
print(out.shape)
print(out)


168
ModulatedAdditiveSkipNet(
  (layers): NetList(
    (0-2): 3 x TorchNet(
      (torch_module): Sequential(
        (0): Linear(in_features=3, out_features=3, bias=True)
        (1): CELU(alpha=1.0)
      )
    )
    (3): TorchNet(
      (torch_module): Sequential(
        (0): Linear(in_features=3, out_features=3, bias=True)
        (1): Identity()
      )
    )
  )
  (connection_modulators[0][0]): TorchNet(
    (torch_module): Identity()
  )
  (connection_modulators[1][0]): TorchNet(
    (torch_module): Linear(in_features=3, out_features=3, bias=True)
  )
  (connection_modulators[1][1]): TorchNet(
    (torch_module): Identity()
  )
  (connection_modulators[2][0]): TorchNet(
    (torch_module): Linear(in_features=3, out_features=3, bias=True)
  )
  (connection_modulators[2][1]): TorchNet(
    (torch_module): Linear(in_features=3, out_features=3, bias=True)
  )
  (connection_modulators[2][2]): TorchNet(
    (torch_module): Identity()
  )
  (connection_modulators[3][0]): TorchNet(
   

In [15]:
90 * 3 + 3 * 3 * 18 + 3 * 3 * 3 + 3 * 3 * 1

468

In [28]:
3 * (90 + 18 + 3 + 1) + 468

804

In [24]:
804

804

In [2]:
from src.networks.core.torch_wrappers.conv_net import Conv2dNet

conv = Conv2dNet(2, 3, 4, padding='same')
conv

Conv2dNet(2, 3, kernel_size=(4, 4), stride=(1, 1), padding=same)

In [3]:
from src.networks.core.torch_wrappers.linear_net import LinearNet

lin = LinearNet(2, 3)
lin(torch.ones(2))

tensor([-0.2231, -0.1688,  1.7347], grad_fn=<ViewBackward0>)